In [1]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
mapbox_access_token = open('mapbox_token.txt').read()

In [3]:
collected = pd.read_csv('Collected.csv')
print(collected.head())

   Unnamed: 0 Location    Latitude, Longitude         Date Name_en Name_ko  \
0           0    Kilju  40.963361, 129.320537  6/1/1911 -     ImOI     임옥인   
1           1  Hamhung  39.991616, 127.612627       1931 -    ImOI     임옥인   
2           2     Nara  34.696509, 135.830573    3/1932 -     ImOI     임옥인   
3           3  Hamhung  39.991616, 127.612627        1935-    ImOI     임옥인   
4           4   Wonsan  39.153670, 127.446306      1937 -     ImOI     임옥인   

    Latitude   Longitude Start_date End_date  
0  40.963361  129.320537   6/1/1911      NaN  
1  39.991616  127.612627       1931      NaN  
2  34.696509  135.830573     3/1932      NaN  
3  39.991616  127.612627       1935      NaN  
4  39.153670  127.446306       1937      NaN  


In [10]:
# create a dictionary of english name : korean name
ko_name = list(collected['Name_ko'].unique())
ko_en_name = {key:value for key, value in zip(en_name, ko_name)}

In [4]:
# Create dictionary of author names
en_name = ['Im Ok-in', 'Shim Hoon', 'Lee Kwang-soo', 'Kim Myung-soon', 'Im Soon-deuk', 'Kim Nam-cheon', 'Mo Yoon-sook', 'Ju Yo-han', 'Lee Sun-hee', 'Cha Mi-ri-sa', 'Yeom Sang-seob', 'No Chun-myeong', 'Kang Kyung-ae', 'Baek Shin-ae', 'Han Mu-sook', 'Park Hwa-seong', 'Choi Jeong-hee', 'Choi Seung-hee', 'Na Hye-seok', 'Lee Sang', 'Kim Sa-ryang', 'Ji Ha-ryeon']
authors_collected = [{'label' : name, 'value': name} for name in en_name]

In [5]:
# Dash App
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)

In [13]:
# app layout using scattermapbox 
app.layout = html.Div([
    html.H1(id = 'author_name_en_ko', children = '20th Century Women Writers Itinerary', style={'text-align': 'center'}),
    html.Div([dcc.Dropdown(
        id = 'author_dropdown',
        value = 'Na Hye-seok',
        multi = True,
        options = authors_collected)]),
    html.Div([
    dcc.Graph(id = 'map', figure = {})]),
    html.Br(),
    html.Button('Next ▶', id='next_button')
    ])

In [ ]:
#for whatever authors are selected, create a list of random colors generated
#make sure that the colors don't overlap
#random color generator? 
#print(index) before the graph with the colors and 
# marker(color - list I'd have created)

In [17]:
# Callback: author_dropdown --> update_graph
@app.callback(
    Output(component_id = 'map', component_property = 'figure'),
    [Input(component_id = 'author_dropdown', component_property = 'value')]
)
def update_graph(author_chosen):
    counts = len(author_chosen)
    author_ko_names = []
    for i in range(len(author_chosen)):
        author_ko_names[i] = ko_en_name.get(author_chosen[i])
    
    #need a for loop 
    df = collected[collected['Name_ko'].isin(author_ko_names)].reset_index()
    
    #fig = go.Figure()
    #for i, row in author_df.iterrows():
    #    fig.add_trace(): 
    fig =go.Figure(go.Scattermapbox(
                    lon = author_df['longitude'],
                    lat = df_sub['latitude'],
                    mode='markers',
                    marker={'color' : author_df['Name_ko'].unique()},
                    #unselected={'marker' : {'opacity':1}},
                    #selected={'marker' : {'opacity':0.5, 'size':25}},
                    hoverinfo='text',
                    hovertext=author_df['Location']
                    #customdata=df_sub['website']
    ))

    # Return figure
    return fig 

In [ ]:
# Callback: author_dropdown --> update_graph
#look at recycle.py (return figure in dictionary form )

In [ ]:
# specify port if error about port number being used already 
if __name__ == '__main__':
    app.run_server(port = 8056)